## Drugs Arrested Case

### ปัจจัยที่ส่งผลต่อจำนวนคดีผู้ต้องหายาเสพติด
#### by กลุ่ม ทุกปัญหามีทางเข้าเสมอ
---

In [1]:
import pandas as pd
import numpy as np
import requests
import json, io, time, urllib3, re
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
urllib3.disable_warnings()

เกริ่นเรื่องปัญหายาเสพติดในไทย

- จำนวนคดีเคส เรียงตามจังหวัด เอาปีล่าสุด - เลือกจังหวัดแสดงผลได้
- ดูเทรนด์ rank top 3 แต่ละปี
- เอาจังหวัดที่น่าสนใจ(สถิติอยู่ในท็อป3บ่อย) ex.นครศรีธรรมราช กรุงเทพ ปทุม สงขลา ขอนแก่น
- ประชากรหนาแน่น ex.กรุงเทพ ขอนแก่น
- ดูเรื่องนักท่องเที่ยว ฟีลเมืองเศรษฐกิจ ex.ชลบุรี
- หัวข้อภาคใต้ เน้นนครศรี + เรื่องกระท่อม + อาชญากรรมอื่น
- การกินเหล้า & เล่นยาสวนทางกัน

#### จำนวนคดีผู้ต้องหาคดียาเสพติด เรียงตามจังหวัด ปี 2565

#### Top 3 Province with Highest Drugs Arrested Case 

#### ปัจจัยความหนาแน่นประชากร

#### ปัจจัยทางเศรษฐกิจ การท่องเที่ยว

#### Southern Province Insights

#### Reversion of Drugs & Alcohol Consumption